In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import StandardScaler

import ast
import pandas as pd
import numpy as np

In [74]:
BATCH_SIZE = 30
EPOCH = 300
HIDDEN_SIZE = 32

In [158]:
df = pd.read_csv("new_dataset.csv")
data_target = np.asarray(df[["data", "target"]])
# np.random.shuffle(data_target)

train_X = data_target[:70, 0]
train_y = data_target[:70, 1]

valid_X = data_target[70:, 0]
valid_y = data_target[70:, 1]

for i in range(train_X.shape[0]):
    train_X[i] = np.asarray(ast.literal_eval(train_X[i]))
    min_val = train_X[i].min(axis=0)
    max_val = train_X[i].max(axis=0)

    train_X[i] = (train_X[i] - min_val) / (max_val - min_val)

for i in range(valid_X.shape[0]):
    valid_X[i] = np.asarray(ast.literal_eval(valid_X[i]))
    min_val = valid_X[i].min(axis=0)
    max_val = valid_X[i].max(axis=0)

    valid_X[i] = (valid_X[i] - min_val) / (max_val - min_val)

train_X = np.array([x for x in train_X])
train_y = np.asarray(train_y.reshape((-1, 1)) / 100, dtype=np.float32)

valid_X = np.array([x for x in valid_X])
valid_y = np.asarray(valid_y.reshape((-1, 1)) / 100, dtype=np.float32)

train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).float()
valid_X = torch.from_numpy(valid_X).float()
valid_y = torch.from_numpy(valid_y).float()


train_X

C:\Users\rayhu\AppData\Local\Temp\ipykernel_25088\2802333248.py:23: RuntimeWarning: invalid value encountered in divide
  valid_X[i] = (valid_X[i] - min_val) / (max_val - min_val)


tensor([[[0.0000, 1.0000, 0.0000],
         [0.0000, 1.0000, 0.0000],
         [0.3333, 0.6515, 0.0000],
         ...,
         [1.0000, 0.0000, 1.0000],
         [1.0000, 0.0000, 1.0000],
         [1.0000, 0.0000, 1.0000]],

        [[0.0000, 1.0000, 1.0000],
         [0.8904, 0.0000, 0.5312],
         [0.9452, 0.2814, 0.6175],
         ...,
         [0.5616, 0.6287, 0.0234],
         [0.5753, 0.5090, 0.0000],
         [0.5753, 0.5090, 0.0000]],

        [[0.0000, 1.0000, 1.0000],
         [0.0000, 1.0000, 1.0000],
         [0.3333, 0.6379, 0.2455],
         ...,
         [1.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000]],

        ...,

        [[1.0000, 1.0000, 1.0000],
         [0.0000, 0.3779, 0.0057],
         [0.0000, 0.3779, 0.0057],
         ...,
         [0.3846, 0.0000, 0.0671],
         [0.3846, 0.0000, 0.0671],
         [0.3846, 0.0000, 0.0671]],

        [[1.0000, 1.0000, 1.0000],
         [0.0000, 0.0392, 0.0000],
         [0.

In [159]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()

        self.x = x
        self.y = y

    def __getitem__(self, index):
        
        x = torch.Tensor(self.x[index])
        y = torch.Tensor(self.y[index])

        return (x, y)
    
    def __len__(self):
        count = self.x.shape[0]
        return count

train_dataset = CustomDataset(train_X, train_y)
valid_dataset = CustomDataset(valid_X, valid_y)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [161]:
class RNN_Model(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = 2

        self.rnn = nn.RNN(
            input_size=3,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            nonlinearity='relu',
            bidirectional=False,
            batch_first=True
        )
        
        self.fc = nn.Sequential(
            nn.Linear(self.hidden_size, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
        

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        x, _ = self.rnn(x)
        y = self.fc(x[:, -1, :])
        return y

In [162]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = RNN_Model(HIDDEN_SIZE).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)


for epoch in range(EPOCH):
    train_loss = 0.0
    valid_loss = 0.0
    predicted_labels = []
    true_labels = []
    
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        print(outputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        train_loss += loss.item() * inputs.size(0)


    model.eval()
    with torch.no_grad():
        for inputs, labels in valid_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)

    print(f"Epoch {epoch}")
    print(f"Average Training Loss: {train_loss / len(train_dataloader.sampler)}")
    print(f"Average Validation Loss: {valid_loss / len(valid_dataloader.sampler)}")
    # print(f"Validation Accuracy: {accuracy_score(true_labels, predicted_labels)}")

    # prfs = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
    
    # print(f"Validation Precision: {prfs[0]}")
    # print(f"Validation Recall: {prfs[1]}")
    # print(f"Validation F1-Score: {prfs[2]}")

    print("==============================================================")    

tensor([[0.0496],
        [0.0496],
        [0.0496],
        [0.0477],
        [0.0488],
        [0.0498],
        [0.0495],
        [0.0496],
        [0.0496],
        [0.0488],
        [0.0478],
        [0.0496],
        [0.0496],
        [0.0461],
        [0.0450],
        [0.0488],
        [0.0497],
        [0.0496],
        [0.0496],
        [0.0521],
        [0.0495],
        [0.0496],
        [0.0498],
        [0.0496],
        [0.0496],
        [0.0497],
        [0.0478],
        [0.0508],
        [0.0476],
        [0.0513]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0500],
        [0.0509],
        [0.0505],
        [0.0529],
        [0.0530],
        [0.0531],
        [0.0505],
        [0.0505],
        [0.0522],
        [0.0505],
        [0.0479],
        [0.0505],
        [0.0498],
        [0.0505],
        [0.0505],
        [0.0504],
        [0.0505],
        [0.0505],
        [0.0506],
        [0.0504],
        [0.0524],
        [0.0505],
        [0.0441],
  

In [163]:
data = torch.tensor([[283, 174, 9.486932754516602], [283, 174, 9.486932754516602], [283, 174, 9.486932754516602], [284, 154, 9.486932754516602], [284, 154, 9.486932754516602], [284, 154, 9.486932754516602], [284, 154, 9.486932754516602], [284, 154, 9.486932754516602], [289, 120, 9.962529182434082], [289, 120, 9.962529182434082]], dtype=torch.float32)
data = nn.functional.normalize(data)

data = data.view(1, 10, 3).to(device)
print(data.shape)
output = model(data)

output * 100

torch.Size([1, 10, 3])


tensor([[59.2294]], device='cuda:0', grad_fn=<MulBackward0>)

In [164]:
for datum in valid_dataloader:
    print(model(datum[0].to(device)) * 100)

tensor([[55.7316],
        [50.3786],
        [58.6183],
        [63.6509],
        [49.5912],
        [64.3808],
        [56.2715],
        [59.9591],
        [50.6802],
        [62.6680],
        [59.5343],
        [63.2404],
        [    nan],
        [51.4824],
        [60.9550]], device='cuda:0', grad_fn=<MulBackward0>)


In [28]:
torch.save(model, "RNN_model.pth")

In [169]:
from joblib import dump, load
dump(ss, 'std_scaler.bin', compress=True)

['std_scaler.bin']